<a href="https://colab.research.google.com/github/Pathway2008/Loan_rating/blob/main/plus_col.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
import pandas as pd
import numpy as np
train = pd.read_csv('/content/drive/MyDrive/loan_rating/train.csv')
test = pd.read_csv('/content/drive/MyDrive/loan_rating/test.csv')
sub = pd.read_csv('/content/drive/MyDrive/loan_rating/sample_submission.csv')

In [45]:
train.drop(['ID'],axis=1,inplace=True)

In [ ]:
train = train[train['주택소유상태'] != 'ANY']
train['근로기간'] = train['근로기간'].replace('< 1 year', '<1 year')
train['근로기간'] = train['근로기간'].replace('1 year', '1 years')
train['근로기간'] = train['근로기간'].replace('3', '3 years')
train['근로기간'] = train['근로기간'].replace('10+years', '10+ years')

In [ ]:
test['근로기간'] = test['근로기간'].replace('< 1 year', '<1 year')
test['근로기간'] = test['근로기간'].replace('1 year', '1 years')
test['근로기간'] = test['근로기간'].replace('3', '3 years')
test['근로기간'] = test['근로기간'].replace('10+years', '10+ years')

In [113]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96293 entries, 0 to 96293
Data columns (total 22 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   대출금액          96293 non-null  int64  
 1   대출기간          96293 non-null  int64  
 2   근로기간          96293 non-null  int64  
 3   주택소유상태        96293 non-null  int64  
 4   연간소득          96293 non-null  int64  
 5   부채_대비_소득_비율   96293 non-null  float64
 6   총계좌수          96293 non-null  int64  
 7   대출목적          96293 non-null  int64  
 8   최근_2년간_연체_횟수  96293 non-null  int64  
 9   총상환원금         96293 non-null  int64  
 10  총상환이자         96293 non-null  float64
 11  총연체금액         96293 non-null  float64
 12  연체계좌수         96293 non-null  float64
 13  대출등급          96293 non-null  int64  
 14  원금/대출         96293 non-null  float64
 15  이자/대출         96293 non-null  float64
 16  대출월금액         96293 non-null  float64
 17  소득/대출         96293 non-null  float64
 18  이자/소득         96293 non-nu

In [75]:
train['원금/대출'] = train['총상환원금']/train['대출금액']
train['이자/대출'] = train['총상환이자']/train['대출금액']

In [112]:
train.head()

,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,...,연체계좌수,대출등급,원금/대출,이자/대출,대출월금액,소득/대출,이자/소득,원금/소득,이자/원금,연체계좌/총계좌
0,12480000,0,6,2,72000000,18.90,15,1,0,0,...,0.0,2,0.000000,0.000000,346666.666667,5.769231,0.000000,0.000000,0.000000,0.0
1,14400000,1,1,0,130800000,22.33,21,10,0,373572,...,0.0,1,0.025943,0.016254,240000.000000,9.083333,0.001789,0.002856,0.626546,0.0
2,12000000,0,5,0,96000000,8.60,14,1,0,928644,...,0.0,0,0.077387,0.012662,333333.333333,8.000000,0.001583,0.009673,0.163619,0.0
3,14400000,0,8,0,132000000,15.09,15,1,0,325824,...,0.0,2,0.022627,0.010632,400000.000000,9.166667,0.001160,0.002468,0.469910,0.0
4,18000000,1,11,2,71736000,25.39,19,8,0,228540,...,0.0,1,0.012697,0.008275,300000.000000,3.985333,0.002076,0.003186,0.651772,0.0


In [83]:
train['대출월금액'] = train.apply(lambda row: row['대출금액'] / 36 if row['대출기간'] == 0 else row['대출금액'] / 60, axis=1)

In [72]:
train['소득/대출'] = train['연간소득'] / train['대출금액']

In [131]:
train['이자/소득'] = train.apply(lambda row: row['총상환이자'] / row['연간소득'] if row['연간소득'] != 0 else 0, axis=1)
train['원금/소득'] = train.apply(lambda row: row['총상환원금'] / row['연간소득'] if row['연간소득'] != 0 else 0, axis=1)

In [96]:
train['이자/원금'] = train.apply(lambda row: row['총상환이자'] / row['총상환원금'] if row['총상환원금'] != 0 else 0, axis=1)

In [111]:
train['연체계좌/총계좌'] = train['연체계좌수'] / train['총계좌수']

In [ ]:
test['원금/대출'] = test['총상환원금'] / test['대출금액']
test['이자/대출'] = test['총상환이자'] / test['대출금액']
test['대출월금액'] = test.apply(lambda row: row['대출금액'] / 36 if row['대출기간'] == 0 else row['대출금액'] / 60, axis=1)
test['소득/대출'] = test['연간소득'] / test['대출금액']
test['이자/소득'] = test.apply(lambda row: row['총상환이자'] / row['연간소득'] if row['연간소득'] != 0 else 0, axis=1)
test['원금/소득'] = test.apply(lambda row: row['총상환원금'] / row['연간소득'] if row['연간소득'] != 0 else 0, axis=1)
test['이자/원금'] = test.apply(lambda row: row['총상환이자'] / row['총상환원금'] if row['총상환원금'] != 0 else 0, axis=1)
test['연체계좌/총계좌'] = test['연체계좌수'] / test['총계좌수']

In [66]:
from sklearn.preprocessing import LabelEncoder
ordinal_features = ['대출기간', '근로기간', '주택소유상태','대출목적']

for feature in ordinal_features:
    le = LabelEncoder()
    le = le.fit(train[feature])
    train[feature] = le.transform(train[feature])

    for label in np.unique(test[feature]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    test[feature] = le.transform(test[feature])

In [77]:
train['대출등급'] = le.fit_transform(train['대출등급'])

In [8]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler

In [114]:
from sklearn.model_selection import train_test_split
X = train.drop(['대출등급'], axis=1)
y = train['대출등급']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size = 0.3, random_state = 42)

In [19]:
from sklearn.model_selection import KFold
kf = StratifiedKFold(n_splits=5, shuffle=False)

In [147]:
xgb_model = xgb.XGBClassifier(objective='multi:softmax', num_class=7, random_state=42)
xgb_model.fit(X_train,y_train)

score = cross_val_score(xgb_model, X_train, y_train, cv=kf, scoring='f1_macro')

In [148]:
score, score.mean()

(array([0.91765625, 0.91219788, 0.9240219 , 0.91108887, 0.91164322]),
 0.9153216228988187)

In [23]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.6 MB/s eta 0:00:00


In [24]:
import xgboost as xgb
from sklearn.neural_network import MLPClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score

In [133]:
from sklearn.model_selection import train_test_split
X = train.drop('대출등급', axis=1)
y = train['대출등급']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size = 0.3, random_state = 42)

In [121]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67405 entries, 42717 to 33408
Data columns (total 21 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   대출금액          67405 non-null  int64  
 1   대출기간          67405 non-null  int64  
 2   근로기간          67405 non-null  int64  
 3   주택소유상태        67405 non-null  int64  
 4   연간소득          67405 non-null  int64  
 5   부채_대비_소득_비율   67405 non-null  float64
 6   총계좌수          67405 non-null  int64  
 7   대출목적          67405 non-null  int64  
 8   최근_2년간_연체_횟수  67405 non-null  int64  
 9   총상환원금         67405 non-null  int64  
 10  총상환이자         67405 non-null  float64
 11  총연체금액         67405 non-null  float64
 12  연체계좌수         67405 non-null  float64
 13  원금/대출         67405 non-null  float64
 14  이자/대출         67405 non-null  float64
 15  대출월금액         67405 non-null  float64
 16  소득/대출         67405 non-null  float64
 17  이자/소득         67405 non-null  float64
 18  원금/소득         67405 no

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Extract the columns to be scaled
columns_to_scale = [col for col in X_train.columns if col not in ['대출기간', '근로기간', '주택소유상태', '대출목적']]

# Create a MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform the selected columns in X_train
X_train_scaled = X_train.copy()
X_train_scaled[columns_to_scale] = scaler.fit_transform(X_train[columns_to_scale])

# Transform the selected columns in X_test
X_test_scaled = X_test.copy()
X_test_scaled[columns_to_scale] = scaler.transform(X_test[columns_to_scale])

In [ ]:
# XGBoost
xgb_model = xgb.XGBClassifier()
xgb_model.fit(X_train_scaled, y_train)
xgb_scores = cross_val_score(xgb_model, X_train_scaled, y_train, cv=kf, scoring='f1_macro')
# print(f'XGBoost Cross-Validation F1 Macro: {xgb_scores.mean()}')
xgb_predictions = xgb_model.predict(X_test_scaled)
xgb_macro = f1_score(y_test, xgb_predictions, average='macro')
# print(f'XGBoost test F1 Macro: {xgb_macro}\n')


# CatBoost
cat_model = CatBoostClassifier(random_state=42, verbose=False)
cat_model.fit(X_train_scaled, y_train)
cat_predictions = cat_model.predict(X_test_scaled)
cat_scores = cross_val_score(cat_model, X_train_scaled, y_train, cv=kf, scoring='f1_macro')
# print(f'CatBoost Cross-Validation F1 Macro: {cat_scores.mean()}')
cat_macro = f1_score(y_test, cat_predictions, average='macro')
# print(f'Cat test F1 Macro: {cat_macro}\n')

# Random Forest
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train_scaled, y_train)
rf_predictions = rf_model.predict(X_test_scaled)
rf_scores = cross_val_score(rf_model, X_train_scaled, y_train, cv=kf, scoring='f1_macro')
# print(f'Random Forest Cross-Validation F1 Macro: {rf_scores.mean()}')
rf_macro = f1_score(y_test, rf_predictions, average='macro')
# print(f'RF test F1 Macro: {rf_macro}\n')

# LightGBM
lgb_model = lgb.LGBMClassifier(random_state=42)
lgb_model.fit(X_train_scaled, y_train)
lgb_predictions = lgb_model.predict(X_test_scaled)
lgb_scores = cross_val_score(lgb_model, X_train_scaled, y_train, cv=kf, scoring='f1_macro')
# print(f'LightGBM Cross-Validation F1 Macro: {lgb_scores.mean()}')
lgb_macro = f1_score(y_test, lgb_predictions, average='macro')
# print(f'RF test F1 Macro: {lgb_macro}\n')

In [135]:
# XGBoost
xgb_model = xgb.XGBClassifier()
xgb_model.fit(X_train, y_train)
xgb_scores = cross_val_score(xgb_model, X_train, y_train, cv=kf, scoring='f1_macro')
# print(f'XGBoost Cross-Validation F1 Macro: {xgb_scores.mean()}')
xgb_predictions = xgb_model.predict(X_test)
xgb_macro = f1_score(y_test, xgb_predictions, average='macro')
# print(f'XGBoost test F1 Macro: {xgb_macro}\n')


# CatBoost
cat_model = CatBoostClassifier(random_state=42, verbose=False)
cat_model.fit(X_train, y_train)
cat_predictions = cat_model.predict(X_test)
cat_scores = cross_val_score(cat_model, X_train, y_train, cv=kf, scoring='f1_macro')
# print(f'CatBoost Cross-Validation F1 Macro: {cat_scores.mean()}')
cat_macro = f1_score(y_test, cat_predictions, average='macro')
# print(f'Cat test F1 Macro: {cat_macro}\n')

# Random Forest
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)
rf_scores = cross_val_score(rf_model, X_train, y_train, cv=kf, scoring='f1_macro')
# print(f'Random Forest Cross-Validation F1 Macro: {rf_scores.mean()}')
rf_macro = f1_score(y_test, rf_predictions, average='macro')
# print(f'RF test F1 Macro: {rf_macro}\n')

# LightGBM
lgb_model = lgb.LGBMClassifier(random_state=42)
lgb_model.fit(X_train, y_train)
lgb_predictions = lgb_model.predict(X_test)
lgb_scores = cross_val_score(lgb_model, X_train, y_train, cv=kf, scoring='f1_macro')
# print(f'LightGBM Cross-Validation F1 Macro: {lgb_scores.mean()}')
lgb_macro = f1_score(y_test, lgb_predictions, average='macro')
# print(f'RF test F1 Macro: {lgb_macro}\n')

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007247 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3284
[LightGBM] [Info] Number of data points in the train set: 67405, number of used features: 21
[LightGBM] [Info] Start training from score -1.747717
[LightGBM] [Info] Start training from score -1.206424
[LightGBM] [Info] Start training from score -1.248802
[LightGBM] [Info] Start training from score -1.975557
[LightGBM] [Info] Start training from score -2.572111
[LightGBM] [Info] Start training from score -3.897369
[LightGBM] [Info] Start training from score -5.434895
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.014222 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3273
[LightGBM] [Info] Number of data points in the train set: 53924, number o

In [26]:
print(f'XGBoost Cross-Validation F1 Macro: {xgb_scores.mean()}')
print(f'Neural Network Cross-Validation F1 Macro: {nn_scores.mean()}')
print(f'CatBoost Cross-Validation F1 Macro: {cat_scores.mean()}')
print(f'Random Forest Cross-Validation F1 Macro: {rf_scores.mean()}')
print(f'LightGBM Cross-Validation F1 Macro: {lgb_scores.mean()}')

XGBoost Cross-Validation F1 Macro: 0.9089996063052805
Neural Network Cross-Validation F1 Macro: 0.3065191161390273
CatBoost Cross-Validation F1 Macro: 0.8902183005400397
Random Forest Cross-Validation F1 Macro: 0.8947749852229687
LightGBM Cross-Validation F1 Macro: 0.8767445402862283


In [31]:
#소득/대출
print(f'XGBoost Cross-Validation F1 Macro: {xgb_scores.mean()}')
print(f'Neural Network Cross-Validation F1 Macro: {nn_scores.mean()}')
print(f'CatBoost Cross-Validation F1 Macro: {cat_scores.mean()}')
print(f'Random Forest Cross-Validation F1 Macro: {rf_scores.mean()}')
print(f'LightGBM Cross-Validation F1 Macro: {lgb_scores.mean()}')

XGBoost Cross-Validation F1 Macro: 0.907585100092698
Neural Network Cross-Validation F1 Macro: 0.34751834673683196
CatBoost Cross-Validation F1 Macro: 0.8890415209018506
Random Forest Cross-Validation F1 Macro: 0.9127493696168608
LightGBM Cross-Validation F1 Macro: 0.8944915699680193


In [37]:
#except 대출기간
print(f'XGBoost Cross-Validation F1 Macro: {xgb_scores.mean()}')
print(f'CatBoost Cross-Validation F1 Macro: {cat_scores.mean()}')
print(f'Random Forest Cross-Validation F1 Macro: {rf_scores.mean()}')
print(f'LightGBM Cross-Validation F1 Macro: {lgb_scores.mean()}')

XGBoost Cross-Validation F1 Macro: 0.9024116815217298
CatBoost Cross-Validation F1 Macro: 0.8859957121131405
Random Forest Cross-Validation F1 Macro: 0.8745078358613979
LightGBM Cross-Validation F1 Macro: 0.888317005507447


In [42]:
#원금/소득, 이자/소득
print(f'XGBoost Cross-Validation F1 Macro: {xgb_scores.mean()}')
print(f'CatBoost Cross-Validation F1 Macro: {cat_scores.mean()}')
print(f'Random Forest Cross-Validation F1 Macro: {rf_scores.mean()}')
print(f'LightGBM Cross-Validation F1 Macro: {lgb_scores.mean()}')

XGBoost Cross-Validation F1 Macro: 0.9078031322506565
CatBoost Cross-Validation F1 Macro: 0.8882495064101402
Random Forest Cross-Validation F1 Macro: 0.8745078358613979
LightGBM Cross-Validation F1 Macro: 0.888317005507447


In [104]:
#이자/원금
print(f'XGBoost Cross-Validation F1 Macro: {xgb_scores.mean()}')
print(f'CatBoost Cross-Validation F1 Macro: {cat_scores.mean()}')
print(f'Random Forest Cross-Validation F1 Macro: {rf_scores.mean()}')
print(f'LightGBM Cross-Validation F1 Macro: {lgb_scores.mean()}')

XGBoost Cross-Validation F1 Macro: 0.915468669651981
CatBoost Cross-Validation F1 Macro: 0.897372382392092
Random Forest Cross-Validation F1 Macro: 0.9150548212303089
LightGBM Cross-Validation F1 Macro: 0.9064435066178493


In [136]:
#연체계좌/총계좌
print(f'XGBoost Cross-Validation F1 Macro: {xgb_scores.mean()}')
print(f'CatBoost Cross-Validation F1 Macro: {cat_scores.mean()}')
print(f'Random Forest Cross-Validation F1 Macro: {rf_scores.mean()}')
print(f'LightGBM Cross-Validation F1 Macro: {lgb_scores.mean()}\n\n')

print(f'xgb test F1 Macro: {xgb_macro}')
print(f'cat test F1 Macro: {cat_macro}')
print(f'rf test F1 Macro: {rf_macro}')
print(f'lgb test F1 Macro: {lgb_macro}')

XGBoost Cross-Validation F1 Macro: 0.9153216228988187
CatBoost Cross-Validation F1 Macro: 0.8985959854332141
Random Forest Cross-Validation F1 Macro: 0.9132654396305304
LightGBM Cross-Validation F1 Macro: 0.8845878572987736


xgb test F1 Macro: 0.9111962985092255
cat test F1 Macro: 0.8968534238360908
rf test F1 Macro: 0.9118980178439041
lgb test F1 Macro: 0.8644472437055299


In [138]:
cat_predictions_flat = np.ravel(cat_predictions)

In [139]:
data = {
    'xgb': xgb_predictions,
    'cat': cat_predictions_flat,
    'lgb': lgb_predictions,
    'rf': rf_predictions,
    'sol': y_test

}

pred_df = pd.DataFrame(data)

In [140]:
rows_with_different_values = pred_df[pred_df.apply(lambda row: row.nunique() > 1, axis=1)]
rows_with_different_values

,xgb,cat,lgb,rf,sol
42625,0,1,1,1,0
33592,1,1,1,2,1
17484,2,1,1,1,1
46409,0,1,1,1,0
73702,4,4,2,3,4
...,...,...,...,...,...
22887,4,2,4,4,2
17856,0,1,0,0,0
62552,2,2,2,2,1
55856,0,1,0,0,0


In [143]:
prediction_columns = ['xgb', 'cat', 'rf', 'lgb']

pred_df['pred'] = pred_df[prediction_columns].mode(axis=1, dropna=False).iloc[:, 0]
pred_df['pred'] = pred_df['pred'].fillna(pred_df['xgb'])

In [145]:
f1_macro_score = f1_score(pred_df['sol'], pred_df['pred'], average='macro')
f1_macro_score

0.9129814209170363

In [144]:
missing_values = pred_df['pred'].isnull().sum()

# Print the result
if missing_values > 0:
    print(f'There are {missing_values} missing values in the \'pred\' column.')
else:
    print('There are no missing values in the \'pred\' column.')

There are no missing values in the 'pred' column.


In [ ]:
#standard scaling
print(f'XGBoost Cross-Validation F1 Macro: {xgb_scores.mean()}')
print(f'CatBoost Cross-Validation F1 Macro: {cat_scores.mean()}')
print(f'Random Forest Cross-Validation F1 Macro: {rf_scores.mean()}')
print(f'LightGBM Cross-Validation F1 Macro: {lgb_scores.mean()}')

In [105]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, StratifiedKFold

# Define the parameter grid for grid search
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2]
}

# Create a grid search object
grid_search = GridSearchCV(xgb_model, param_grid, scoring='f1_macro', cv=kf)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Display the best parameters and corresponding f1_macro score
print("Best Parameters: ", grid_search.best_params_)
print("Best F1-macro Score: {:.4f}".format(grid_search.best_score_))

Best Parameters:  {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 100}
Best F1-macro Score: 0.9165


In [106]:
#cat
param_grid = {
    'iterations': [100, 200, 300],
    'depth': [4, 6, 8],
    'learning_rate': [0.01, 0.1, 0.2]
}

# Create a grid search object
grid_search_cat = GridSearchCV(cat_model, param_grid, scoring='f1_macro', cv=kf)

# Fit the grid search to the data
grid_search_cat.fit(X_train, y_train)

# Display the best parameters and corresponding f1_macro score
print("Best Parameters: ", grid_search_cat.best_params_)
print("Best F1-macro Score: {:.4f}".format(grid_search_cat.best_score_))

Best Parameters:  {'depth': 8, 'iterations': 300, 'learning_rate': 0.2}
Best F1-macro Score: 0.9008
